EDA:

Let's begin by importing the necessary libraries and loading the dataset. Since our dataset is massive, and has grown by 25% in just the last 2 years, let's create a subset of the data to work with. Let's analyze the data from 2020 onwards for a more manageable dataset and to observe Covid-19's impact on the liquor sales. We start by filtering the data for dates from 2020 onwards and implementing an multi index structure to optimize performance for our product and time series analysis. This is a one time operation, since loading the original dataset is time consuming, so we will save the output as a parquet file and start our EDA on the parquet file.


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
liquor_sales = pd.read_csv('Iowa_Liquor_Sales.csv')

liquor_sales['Date'] = pd.to_datetime(liquor_sales['Date'])

# Filter for dates from 2020 onwards
filtered_df = liquor_sales[liquor_sales['Date'].dt.year >= 2020].copy()



# Sort values first for better performance
filtered_df_sorted = filtered_df.sort_values(['Date', 'Item Number', 'Store Number'])

# Create multi-index structure optimized for product analysis
filtered_df_indexed = filtered_df_sorted.set_index(['Date', 'Item Number', 'Store Number'])

# Export to parquet with optimized settings
output_filename = 'iowa_liquor_sales_2020_onwards.parquet'
filtered_df_indexed.to_parquet(
    output_filename,
    engine='pyarrow',
    compression='snappy'
)



Creating 1GB subset...


C:\Users\haide\AppData\Local\Temp\ipykernel_22076\4035682179.py:38: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunk_size):
C:\Users\haide\AppData\Local\Temp\ipykernel_22076\4035682179.py:38: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunk_size):
C:\Users\haide\AppData\Local\Temp\ipykernel_22076\4035682179.py:38: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunk_size):
C:\Users\haide\AppData\Local\Temp\ipykernel_22076\4035682179.py:38: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunk_size):
C:\Users\haide\AppData\Local\Temp\ipykernel_22076\4035682179.py:38: DtypeWarning

Finalizing the subset...


C:\Users\haide\AppData\Local\Temp\ipykernel_22076\4035682179.py:53: DtypeWarning: Columns (6,14) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv(output_file)



Final file size: 1.00GB
Number of rows in subset: 4,300,000
Date range: from 2012-01-03 to 2022-09-30
